In [1]:
import dspy

g:\Mi unidad\SONY-GATEWAY\MaLB-SC-generation-module\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gpt4_turbo = dspy.OpenAI(model='gpt-4-1106-preview', max_tokens=300)
gpt3_turbo = dspy.OpenAI(model='gpt-3.5-turbo-1106', max_tokens=300, temperature=1)

dspy.settings.configure(lm=gpt3_turbo, max_tokens=1024)

In [3]:
A1 = """
When a buyer requests a refund, the smart contract should compensate them fairly. To achieve this, the contract needs to consider the buyer and the current state of the sale, such as whether it's in progress or cancelled. Only the seller is authorized to initiate the refund process, ensuring that refunds are legitimate and secure. In the primary scenario, if the sale is in progress, the buyer should receive a refund of twice the value they paid initially if the seller decides not to proceed with the sale. However, if the sale is not in progress, no refund will be issued. This requirement ensures that buyers are fairly compensated in case of a sale cancellation, while also protecting sellers from unauthorized refunds
"""

A2 = """
A smart contract is a self-executing contract with the terms of the agreement between buyer and seller being directly written into lines of code. The code and the agreements contained therein exist across a distributed, decentralized blockchain network. The code controls the execution, and transactions are trackable and irreversible.
"""

A3 = """
I have a blockchain 3 business and I want to improve the efficiency of it by implementing smart contracts.
"""

In [4]:
import pydantic
main_description = A1

class IsAboutSmartContracts(pydantic.BaseModel):
    parameter: bool

class AssessTopic(dspy.Signature):
    """Verify that the description talks about Smart Contracts"""
    description: str = dspy.InputField(desc="A description of a Smart Contract")
    output: IsAboutSmartContracts = dspy.OutputField(desc="True/False indicating if text is about Smart Contracts")

class VerifySmartContractTopic(dspy.Module):
    """A module to verify if the description is about Smart Contracts"""
    def __init__(self):
        super().__init__()
        self.predictor = dspy.functional.TypedPredictor(AssessTopic)

    def forward(self, description: str) -> IsAboutSmartContracts:
        prediction = self.predictor(description=description)
        return prediction.output.parameter

# Example usage
verifier = VerifySmartContractTopic()
result = verifier.forward(main_description)
print(result)

True


In [5]:
import pydantic
description = A1

# Pydantic model to define the output structure
class OutputRate(pydantic.BaseModel):
    parameter: int

class Rate(dspy.Signature):
    """Rate the technical completeness of the smart contract description on a scale of 1 to 10"""
    description: str = dspy.InputField(desc="A description of a Smart Contract")
    output: OutputRate = dspy.OutputField(desc="A rating from 1 to 10")

class RateModule(dspy.Module):
    """A module to rate the technical completeness of a smart contract description"""
    def __init__(self):
        super().__init__()
        self.predictor = dspy.functional.TypedPredictor(Rate)

    def rate_description(self, description: str) -> int:
        prediction = self.predictor(description=description)
        return prediction.output.parameter

# Example usage of the module
rate_module = RateModule()
rating = rate_module.rate_description(main_description)
print(rating)

7


In [8]:
import pydantic
from typing import List
description = A1

class RequirementsList(pydantic.BaseModel):
    requirements: List[str]

class extract_requirements(dspy.Signature):
    """Extract requirements from a Smart Contract Description"""
    description: str = dspy.InputField(desc="A description of a Smart Contract")
    output: RequirementsList = dspy.OutputField(desc="A list object with extracted requirements")

class ExtractRequirements(dspy.Module):
    """A module to extract requirements from a Smart Contract Description"""
    def __init__(self):
        super().__init__()
        self.predictor = dspy.functional.TypedPredictor(extract_requirements)

    def forward(self, description: str) -> RequirementsList:
        prediction = self.predictor(description=description)
        return prediction.output.requirements

req_module = ExtractRequirements()
list_of_requirements = req_module.forward(main_description)
print(list_of_requirements)


ImportError: cannot import name 'List' from 'pydantic' (g:\Mi unidad\SONY-GATEWAY\MaLB-SC-generation-module\.venv\Lib\site-packages\pydantic\__init__.py)

In [202]:
import pydantic
from typing import List

class Requirement(pydantic.BaseModel):
    Name: str
    Scope: str
    Input: List[str]
    Constraints: str
    Output: str
    PrimaryScenario: str
    AlternativeScenario: str

class generate_attributes(dspy.Signature):
    """Generate attributes for the given attribute description."""
    context: str = dspy.InputField(desc="Context of the requirement")
    requirement: str = dspy.InputField(desc="Requirement description")
    output: Requirement = dspy.OutputField(desc="Structured list of requirement attributes")

class GenerateAttributes(dspy.Module):
    """A module to process multiple requirement descriptions into structured JSON."""

    def __init__(self):
        super().__init__()
        self.predictor = dspy.functional.TypedPredictor(generate_attributes)
    
    def forward(self, context: str, requirement: str) -> Requirement:
        prediction = self.predictor(context=context, requirement=requirement)
        return prediction.output

req_module = GenerateAttributes()
# List of requirements has multiple strings inside.
for req in list_of_requirements:
    attributes = req_module.forward(main_description, req)
    print(attributes.dict())

{'Name': 'CompensateBuyers', 'Scope': 'Refund process', 'Input': ['Buyer request', 'Sale status'], 'Constraints': 'Only seller can initiate refund process', 'Output': 'Fair compensation for buyers', 'PrimaryScenario': 'If sale in progress, buyer receives twice the initial value as refund if seller cancels sale. If sale not in progress, no refund issued.', 'AlternativeScenario': 'None'}


{'Name': 'Consider the current state of the sale when processing refunds', 'Scope': 'Refund process for buyers in a sale', 'Input': ['Buyer request for refund', 'Current state of the sale (in progress or cancelled)'], 'Constraints': 'Only seller is authorized to initiate the refund process', 'Output': 'Fair compensation for buyers in case of sale cancellation', 'PrimaryScenario': 'If sale is in progress, buyer receives a refund of twice the value they paid if seller decides not to proceed with the sale', 'AlternativeScenario': 'If sale is not in progress, no refund will be issued'}
{'Name': 'Authorization for refund initiation', 'Scope': 'Seller authorization', 'Input': ['Buyer request', 'Sale status'], 'Constraints': 'Only seller is authorized', 'Output': 'Refund initiation', 'PrimaryScenario': 'Sale in progress: Buyer receives double the initial payment if sale is cancelled', 'AlternativeScenario': 'Sale not in progress: No refund issued'}
{'Name': 'Refund Compensation', 'Scope': 'Sm